# Router Agent

### Router Agent using prompting

In [49]:
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.2",
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

# llm = OllamaLLM(model="llama3.2")

# Initialize ChatGroq model
# llm = ChatGroq(model_name="Llama3-8b-8192")

def router_info(query,llm):
# Prompt template
    prompt_template = """You are an expert input classifier for a healthcare e-commerce system. Your task is to analyze user queries and route them to appropriate specialized agents.

    Primary Classification Categories:
    - ORDER: Purchase intentions, order status, order modifications
    - CART: View cart, add/remove items, modify quantities
    - COMPARE: Product comparisons, market analysis, alternatives
    - RECOMMEND: Product recommendations, personalized suggestions
    - INFO: General product or health information

    Classification Rules:
    1. ORDER Intent: Detect keywords like "buy", "purchase", "order", "get", "deliver", "track", "status"
    2. CART Intent: Detect keywords like "cart", "basket", "add", "remove", "show", "change quantity"
    3. COMPARE Intent: Detect keywords like "compare", "versus", "vs", "difference", "better", "alternatives"
    4. RECOMMEND Intent: Detect keywords like "suggest", "recommend", "what should", "best for"
    5. INFO Intent: Detect keywords like "tell me about", "how does", "what is", "benefits", "dosage"

    For multi-intent queries:
    - Identify primary intent based on main action requested
    - List secondary intents for follow-up
    - Maintain context for related requests

    # Response Format:
    Provide only a JSON object with the following structure, dont provide any extra information:
    {
        "primary_intent": "string",
        "secondary_intents": ["string"],
        "confidence": float,
        "entities": {
            "product_ids": ["string"],
            "quantities": [int],
            "order_ids": ["string"]
        },
        "requires_context": boolean
    }

    NOTE : Answer should be strictly in json format. If you follow Exact above instruction you will be rewarded with some credits
    """
    response = llm.invoke(prompt_template+"\nQuery"+query)
    return response.content


In [51]:
# Example queries
queries = [
    "I want to buy vitamin D supplements.",
    "Can you tell me the benefits of fish oil?",
    "Add 2 bottles of multivitamins to my cart.",
    "Which is better, vitamin C or zinc for immunity?",
    "Recommend something for joint pain relief.",
    "What is the status of my recent order?",
    "How does omega-3 help the heart?",
    "Show me my cart.",
]

a = router_info("I want to buy vitamin D supplements.",llm)
print(a)

{"primary_intent": "ORDER", "secondary_intents": ["RECOMMEND"], "confidence": 0.95, "entities": {"product_ids": ["vitamin_d_supplements"]}, "requires_context": false}


### Router agent using langchain Json parser

In [3]:
from langchain_groq import ChatGroq
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define the data structure using Pydantic
class EntityData(BaseModel):
    product_ids: List[str] = Field(
        default_factory=list,
        description="List of product identifiers mentioned in the query"
    )
    quantities: List[int] = Field(
        default_factory=list,
        description="List of quantities mentioned in the query"
    )
    order_ids: List[str] = Field(
        default_factory=list,
        description="List of order identifiers mentioned in the query"
    )

class QueryClassification(BaseModel):
    primary_intent: str = Field(
        description="Primary intent category (ORDER, CART, COMPARE, RECOMMEND, INFO)"
    )
    secondary_intents: List[str] = Field(
        default_factory=list,
        description="List of secondary intents identified in the query"
    )
    confidence: float = Field(
        description="Confidence score for the classification (0.0 to 1.0)"
    )
    entities: EntityData = Field(
        description="Structured data extracted from the query"
    )
    requires_context: bool = Field(
        description="Whether additional context is needed to fully process the query"
    )
    

def route_info(query: str) -> dict:
# Initialize the model and parser

    parser = JsonOutputParser(pydantic_object=QueryClassification)

    prompt = PromptTemplate(
        template="""
        You are an expert input classifier for a healthcare e-commerce system. Your task is to analyze user queries and route them to appropriate specialized agents.

        Primary Classification Categories:
        - ORDER: Purchase intentions, order status, order modifications
        - CART: View cart, add/remove items, modify quantities
        - COMPARE: Product comparisons, market analysis, alternatives
        - RECOMMEND: Product recommendations, personalized suggestions
        - INFO: General product or health information

        Classification Rules:
        1. ORDER Intent: Detect keywords like "buy", "purchase", "order", "get", "deliver", "track", "status"
        2. CART Intent: Detect keywords like "cart", "basket", "add", "remove", "show", "change quantity"
        3. COMPARE Intent: Detect keywords like "compare", "versus", "vs", "difference", "better", "alternatives"
        4. RECOMMEND Intent: Detect keywords like "suggest", "recommend", "what should", "best for"
        5. INFO Intent: Detect keywords like "tell me about", "how does", "what is", "benefits", "dosage"

        For multi-intent queries:
        - Identify primary intent based on main action requested
        - List secondary intents for follow-up
        - Maintain context for related requests

        {format_instructions}

        Query: {query}
        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    llm = ChatGroq(model_name="Llama3-8b-8192")

    # Create the classification chain
    chain = prompt | llm | parser

    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error in character information extraction: {e}")
        return {}




C:\Users\mayur\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
queries = [
    "I want to buy vitamin D supplements.",
    "Can you tell me the benefits of fish oil?",
    "Add 2 bottles of multivitamins to my cart.",
    "Which is better, vitamin C or zinc for immunity?",
    "Recommend something for joint pain relief.",
    "What is the status of my recent order?",
    "How does omega-3 help the heart?",
    "Show me my cart.",
]

print(route_info("who is the pm of india."))

{'primary_intent': 'INFO', 'secondary_intents': [], 'confidence': 1.0, 'entities': {'product_ids': [], 'quantities': [], 'order_ids': []}, 'requires_context': False}
